# タスク 5: コード生成のために Bedrock モデルを呼び出す

このノートブックでは、大規模言語モデル (LLM) を使用してテキストプロンプトに基づいてコードを生成する方法を学習します。

このノートブックで使用されるプロンプトはゼロショットプロンプトです。プロンプト自体以外のテキストの例は提供していません。

Amazon Bedrock のモデルのコード生成機能を示すために、コード生成のユースケースを取り上げます。Boto3 クライアントを使用して Amazon Bedrock API と通信します。タスク、命令、およびモデルへ渡される入力を API へ提供して、追加の例を提供せずに出力を生成する方法を探ります。ここでの目的は、強力な LLM が手元のタスクを簡単に理解し、魅力的な出力を生成する方法を示すことです。



## シナリオ

あなたは AnyCompany のデータ アナリストである Moe です。この会社は、過去 1 年間のさまざまな製品の販売実績を把握したいと考えています。sales.csv というデータセットが提供されています。データセットには次の列が含まれています:

- 日付 (YYYY-MM-DD) 形式
- Product_ID (各製品の一意の識別子)
- 価格 (各製品の販売価格)


このノートブックでは、特定のプロンプトのコードを生成する方法を学習します。Boto3 クライアントで Amazon Bedrock API を使用して Amazon Nova Lite モデルを使用します。

## Task 5.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## タスク 5.2: コード生成

このタスクでは、Amazon Bedrock サービスの入力を準備して、ユースケース用の Python プログラムを生成します。

#### ラボのセットアップ - このラボのサンプル sales.csv データを作成します。



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## タスク 5.3: Amazon Bedrock で生成された Python プログラムで売上を分析する

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

CSV、sales.csv があり、列は次のようになっています:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

CSV ファイルから売上データを分析する Python プログラムを作成してください。プログラムはデータを読み取り、以下を算出・表示できる必要があります:

- 年間の総売上
- 製品別の総売上
- 売上が最も高い製品
- 売上が最も高い日付とその日付に達成された売上
- 棒グラフを使用して月毎の売上を視覚化

Python プログラムを作成する際には、下記も考慮して下さい。

- コードが構文的に正しく、バグがなく、最適化されており、不必要に複数行にまたがっていないようにしてください。
- 標準ライブラリを使用して下さい。
- 周囲のテキスト、説明、またはコンテキストなしで、Python コードのみを回答して下さい。

"""

In [ ]:
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 8192,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}) 

## タスク 5.4: モデルの呼び出し

In [ ]:
modelId = "amazon.nova-lite-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
#output_list = response_body.get('content', [])[0].get("text", "")
output_list = response_body['output']['message']['content'][0]['text']
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i> (オプション) 印刷された出力から生成されたコードをコピーし、検証のために下のセルで Bedrock によって生成されたコードを実行します。


これで、Amazon Bedrock API を使用する `boto3` SDK を試しました。この API を使用して、特定の販売データを分析および視覚化する Python プログラムを生成します。



### 試してみましょう

- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、**タスク 6** に進んでください。